In [1]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

env: CUDA_VISIBLE_DEVICES=0


In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from skimage.measure import compare_psnr
import tensorflow as tf
from data.dataset import Dataset
import data.data_selector as ds
import utils.data_processing as dp
import utils.plot_functions as pf
import analysis.analysis_picker as ap
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import matplotlib.pyplot as plt

## Load analyzers

In [4]:
print([name[17:-4] for name in os.listdir(os.path.expanduser("~/Work/Projects/lca_subspace_vh/logfiles")) if name[17:19] == "5x"])
versions = [['5x_2_1.0_0.2', '5x_2_1.5_0.2', '5x_2_2.0_0.2'],
            ['5x_4_1.0_0.2', '5x_4_1.5_0.2', '5x_4_2.0_0.2'],
            ['5x_5_1.0_0.2', '5x_5_1.5_0.2', '5x_5_2.0_0.2'],
            ['5x_8_1.0_0.2', '5x_8_1.5_0.2', '5x_8_2.0_0.2']]
versions = np.array(versions)
# versions = ['5x_4_1.0_0.2', '5x_4_1.25_0.2', '5x_4_1.75_0.2', '5x_4_2.0_0.2']
num_images = 10
num_patches = 40
colors = ["red", "blue", "green", "black"] ## should have same length as versions

['5x_3_2.0_0.2', '5x_4_2.0_0.2', '5x_5_1.0_0.3', '5x_5_1.0_0.2', '5x_8_2.0_0.2', '5x_4_1.5_0.2', '5x_8_1.0_0.2', '5x_3_1.0_0.2', '5x_2_1.5_0.2', '5x_2_2.0_0.2', '5x_4_1.75_0.2', '5x_2_1.0_0.2', '5x_4_2.0_0.3', '5x_4_1.0_0.2', '5x_4_3.0_0.3', '5x_4_1.25_0.2', '5x_5_1.5_0.2', '5x_4_1.0_0.1', '5x_5_1.0_0.1', '5x_3_1.5_0.2', '5x_4_0.5.0_0.2', '5x_8_1.5_0.2', '5x_2_1.0_0.3', '5x_4_1.0_0.3', '5x_4_2.0_0.1', '5x_5_2.0_0.2']


In [5]:
analyzers = {} 
for version in versions.flatten():
    print(f"Creating analyzer with version {version}")
    class params:
        def __init__(self):
            self.model_type = "lca_subspace"
            self.model_name = "lca_subspace_vh"
            self.version = version
    #         self.version = "10x_4_3.0_0.3"
            self.save_info = "analysis_train"
            self.overwrite_analysis_log = False

    analysis_params = params()
    analysis_params.project_dir = (os.path.expanduser("~")+"/Work/Projects/")
    analysis_params.model_dir = (analysis_params.project_dir+analysis_params.model_name)
    
    analyzer = ap.get_analyzer(analysis_params.model_type)
    analyzer.setup(analysis_params)
    analyzer.setup_model(analyzer.model_params)
#    analyzer.load_analysis(save_info=analysis_params.save_info)
    analyzer.model_name = analysis_params.model_name
    
    analyzers[version] = analyzer

Creating analyzer with version 5x_2_1.0_0.2
Creating analyzer with version 5x_2_1.5_0.2
Creating analyzer with version 5x_2_2.0_0.2
Creating analyzer with version 5x_4_1.0_0.2
Creating analyzer with version 5x_4_1.5_0.2
Creating analyzer with version 5x_4_2.0_0.2
Creating analyzer with version 5x_5_1.0_0.2
Creating analyzer with version 5x_5_1.5_0.2
Creating analyzer with version 5x_5_2.0_0.2
Creating analyzer with version 5x_8_1.0_0.2
Creating analyzer with version 5x_8_1.5_0.2
Creating analyzer with version 5x_8_2.0_0.2


## other functions

In [6]:
def get_full_images(analyzer, num_images):
    class img_params():
        def __init__(self, num_images):
            self.data_type = analyzer.model_params.data_type
            self.num_images = num_images
            self.extract_patches = False
            self.image_edge_size = 128
            self.data_dir = os.path.expanduser("~")+"/Work/Datasets/"
            self.rand_seed = 10
            self.rand_state = np.random.RandomState(self.rand_seed)
    #         self.rand_seed = analyzer.model_params.rand_seed
    #         self.rand_state = np.random.RandomState(analysis_params.rand_seed)
    dataset = ds.get_data(img_params(num_images))
    full_image = dp.reshape_data(dataset["train"].images[:num_images], flatten=False)[0]
    return full_image

In [7]:
def get_recon_images(analyzer, full_image):
    if analyzer.model_params.whiten_data:
      # FT method is the only one that works on full images
      wht_img, img_mean, ft_filter = dp.whiten_data(full_images,
        method="FT", lpf_cutoff=analyzer.model_params.lpf_cutoff)
    else:
      wht_img = full_image
    img_patches = dp.extract_patches(wht_img,
      out_shape=(1, analyzer.model_params.patch_edge_size, analyzer.model_params.patch_edge_size, 1),
      overlapping=False, randomize=False, var_thresh=0.0)
    img_patches, orig_shape = dp.reshape_data(img_patches, flatten=True)[:2]
    model_eval = analyzer.evaluate_model(img_patches,
      ["inference/activity:0", "output/reconstruction:0"])
    recon_patches = model_eval["output/reconstruction:0"]
    a_vals = model_eval["inference/activity:0"]
    recon_patches = dp.reshape_data(recon_patches, flatten=True)[0]#, out_shape=orig_shape)[0]
    return img_patches, recon_patches, a_vals
    # full_recon = dp.patches_to_image(recon_patches, full_image.shape).astype(np.float32)
    # if analyzer.model_params.whiten_data:
    #   full_recon = dp.unwhiten_data(full_recon, img_mean, ft_filter, method="FT")

def get_normed_images(full_image, full_recon):
    normed_image = (
      (full_image - np.min(full_image))
      / (np.max(full_image) - np.min(full_image))).astype(np.float32)

    normed_recon = (
      (full_recon - np.min(full_recon))
      / (np.max(full_recon) - np.min(full_recon))).astype(np.float32)
    return normed_image, normed_recon

def mean_squared_error(img, recon):
    return np.mean(np.power(img - recon, 2), axis=1)

def compute_recon_frac_act(a_vals):
    return np.count_nonzero(a_vals, axis=1) / a_vals[0].size

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

## Generate Scatter Plot

In [6]:
full_images = get_full_images(analyzer, num_images)

NameError: name 'get_full_images' is not defined

In [25]:
fig, ax = plt.subplots(1, 1, figsize=(8,8), dpi=1000)
# ax.set_title('Recon Loss vs Fraction Active')
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
# ax.set_xlim([0.0, 1.0])
# ax.set_ylim([0.0, np.ceil(ymax+ymax*0.1)])

versions = [['5x_2_1.0_0.2', '5x_2_1.5_0.2', '5x_2_2.0_0.2'],
            ['5x_4_1.0_0.2', '5x_4_1.5_0.2', '5x_4_2.0_0.2'],
            ['5x_5_1.0_0.2', '5x_5_1.5_0.2', '5x_5_2.0_0.2'],
            ['5x_8_1.0_0.2', '5x_8_1.5_0.2', '5x_8_2.0_0.2']]
versions = np.array(versions)
# versions = [['5x_4_2.0_0.2']]
colors = ["red", "blue", "green", "black"]
markers = {"1.0":"^", "1.5":"s", "2.0":"o"}
# versions = np.array([['5x_2_2.0_0.2']])

for g, gs in enumerate(versions):
    # generate mse for fixed group size
    group_mse = []
    group_recon_frac = []
    for v, version in enumerate(versions[g]):
        _, group_size, lambda_multi, _ = version.split("_")
        stat_dict = analyzers[version].stats_analysis(analysis_params.save_info)
            # confidence ellipse
#         confidence_ellipse(np.array(stat_dict["a_fraction_active"]), np.array(stat_dict["recon_loss"]), ax, n_std=1, edgecolor=colors[g])
            # group mse and recon frac
#         plt.scatter(np.array(stat_dict["a_fraction_active"]), np.array(stat_dict["recon_loss"]), c=colors[g], s=2)
        ax.scatter(np.array(stat_dict["a_fraction_active"]), np.array(stat_dict["recon_loss"]), marker=markers[lambda_multi], edgecolor=None, alpha=0.6, s=90, c=colors[g], lw=0, label=r"$N = ${}, $\lambda = ${}".format(group_size, lambda_multi))
legend = ax.legend(loc="upper right", borderaxespad=0., borderpad=0., framealpha=0.0, fontsize=14, markerscale=1.2)
ax.tick_params(axis='both', labelsize=14)
ax.set_xlabel('Fraction of Neurons Active', fontsize=17)
ax.set_ylabel('Reconstruction Loss', fontsize=17)
fig.tight_layout()
plt.savefig(os.path.expanduser("~")+"/ssc_param_sweep.png", bbox_inches='tight')

## reconstruction of the whole image

In [ ]:
analyzer = analyzers["5x_4_1.0_0.2"]
full_image = get_full_images(analyzer, num_images)

In [ ]:
for v in ["5x_4_1.0_0.2", "5x_4_1.5_0.2", "5x_4_2.0_0.2"]:
    analyzer = analyzers[v]
    if analyzer.model_params.whiten_data:
      # FT method is the only one that works on full images
      wht_img, img_mean, ft_filter = dp.whiten_data(full_image,
        method="FT", lpf_cutoff=analyzer.model_params.lpf_cutoff)
    else:
      wht_img = full_image
    img_patches = dp.extract_patches(wht_img,
      out_shape=(1, analyzer.model_params.patch_edge_size, analyzer.model_params.patch_edge_size, 1),
      overlapping=False, randomize=False, var_thresh=0.0)
    img_patches, orig_shape = dp.reshape_data(img_patches, flatten=True)[:2]
    model_eval = analyzer.evaluate_model(img_patches,
      ["inference/activity:0", "output/reconstruction:0"])
    recon_patches = model_eval["output/reconstruction:0"]
    a_vals = model_eval["inference/activity:0"]
    recon_patches = dp.reshape_data(recon_patches, flatten=False, out_shape=orig_shape)[0]
    full_recon = dp.patches_to_image(recon_patches, full_image.shape).astype(np.float32)
    if analyzer.model_params.whiten_data:
        full_recon = dp.unwhiten_data(full_recon, img_mean, ft_filter, method="FT")
    ncols=7
    fig, ax = plt.subplots(ncols=ncols, nrows=2, figsize=(ncols, 2))
    for c in range(ncols):
        ax[0, c].imshow(full_recon[c, :, :, 0], cmap="gray");ax[0, c].axis("off")
        ax[1, c].imshow(full_image[c, :, :, 0], cmap="gray");ax[1, c].axis("off")
    plt.show()


In [ ]:
i = 9
plt.imshow(full_recon[1, :, :, 0], cmap="gray"); plt.show()
plt.imshow(full_image[1, :, :, 0], cmap="gray");plt.show()
plt.imshow(full_recon[3, :, :, 0], cmap="gray"); plt.show()
plt.imshow(full_image[3, :, :, 0], cmap="gray");plt.show()
plt.imshow(full_recon[7, :, :, 0], cmap="gray"); plt.show()
plt.imshow(full_image[7, :, :, 0], cmap="gray");plt.show()
plt.imshow(full_recon[9, :, :, 0], cmap="gray"); plt.show()
plt.imshow(full_image[9, :, :, 0], cmap="gray");plt.show()